In [1]:
from numpy import dot
from numpy.linalg import norm
import openai
import copy
import random
random.seed(4)

In [2]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
def get_response(prompt=None, content=None):
    openai.api_type = "azure"
    openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
    openai.api_version = "2023-03-15-preview"
    # key1: 19ea901e8e10475da1bb0537abf8e5a4
    # key2: 548e5c0c2aff453e932948927a27bde6
    openai.api_key = "19ea901e8e10475da1bb0537abf8e5a4"

    # role : system|user|assistant
    gpt_config = {'engine': 'gpt-35-turbo',
                'role': 'user',
                }
    message_list = [
            {"role": "system",
            "content": f"{content}"},
            {"role": "user", "content": f"{prompt}"}
            ]
    response = openai.ChatCompletion.create(
        engine=gpt_config['engine'],
        temperature=0.6,
        messages=message_list
    )
    return response['choices'][0]['message']['content']

### 定义embedding获取函数
def get_embedding(text, model="text-embedding-ada-002"):
    openai_api_key = "548e5c0c2aff453e932948927a27bde6"
    openai.api_key = openai_api_key
    openai.api_type = "azure"
    # openai.api_version = "2023-06-15-preview"
    openai.api_version = "2023-03-15-preview"
    openai.api_base = "https://bigo-chatgpt-9.openai.azure.com/"
    res = openai.Embedding.create(
    input=[text], deployment_id="text-embedding-ada-002")['data'][0]['embedding']
    return res

## 制作沙盒

In [3]:
world_dic = {0:'森林',   
                1:'港口小镇',
                2:'乡村',
                3:'沙漠',
                4:'都市',
                5:'大海'}
def make_maze():
    ### 简单定义一些对应的场景
    maze = [[dict() for i in range(10)] for j in range(10)]

    ### 定义一些场景，并为每个场景加入一个随机事件
    event_prompt = ("请根据场景：{world}\n"
                    "生成一个随机事件。用一段话描述或概括该随机事件。讲述事件的主要人物，经过，结果。只返回事件内容。")
    for i in range(len(maze)):
        for j in range(len(maze[0])):
            maze[i][j]['world'] = random.randrange(0, 6)
            if 'events' in maze[i][j]:
                event = get_response(event_prompt.format_map({'world':world_dic[maze[i][j]['world']]}))
                maze[i][j]['events'].append(event)
            else:
                maze[i][j]['events'] = []
                event = get_response(event_prompt.format_map({'world':world_dic[maze[i][j]['world']]}))
                maze[i][j]['events'].append(event)
    return maze

## 定义人格（张三、李四）

In [4]:
### 定义persona大类
class Persona():
    def __init__(self, name, persona, skills, interests, associative_memory=None):
        self.name = name
        self.persiona = persona
        self.skills = skills
        self.interests = interests
        self.asso_mem = associative_memory if associative_memory is not None else None

    def get_basic_info_prompt(self):
        prompt_template = (
            "你是沙盒世界的一员，你的名字是{name}。\n"
            "以下是你的性格:\n"
            "{persona}\n\n"
            "你掌握的技巧:\n"
            "{skills}\n\n"
            "你的兴趣爱好: {interests}\n"
        )
        return prompt_template.format_map({'name':self.name, 'persona':self.persiona, 'skills':self.skills, 'interests':self.interests})
    
    def get_summarized_asso_mem(self):
        if self.asso_mem is None or self.asso_mem is []:
            print("还没有历史记忆")
            return 
        else:
            events_prompt = ''
            for i in range(len(self.asso_mem)):
                event = self.asso_mem[i]
                events_prompt = events_prompt + f'事件: {event}'
            summarize_prompt = '请总结这些过往的事件:' + events_prompt + '每个事件用一句话去总结'
            #print(summarize_prompt)
            summarized_asso_mem = get_response(summarize_prompt)
            summarized_events = [eve for eve in summarized_asso_mem.split('\n') if eve != ""]
            return summarized_events
        
### 现在初始化几个人格
zhangsan_dic = {'name': "张三",
                'persona': "温和友善： 张三是一个亲切友好的人，总是愿意帮助他人，因此在社交圈中备受喜爱。\n勤奋和有责任感： 他非常注重工作，总是尽最大努力完成任务。他喜欢有组织和计划的生活，这使他在工作和生活中都表现出色。\n耐心： 张三拥有出色的耐心，不易冲动，能够冷静地应对压力和困难。\n好奇心旺盛： 他对世界充满好奇，喜欢学习新事物，经常探索新的兴趣领域。",
                'skills': "沟通技巧： 张三是一位出色的沟通者，擅长倾听他人并表达自己的观点，这使他在职业和社交场合都表现出色。\n烹饪： 他对烹饪非常有天赋，经常烹制美味的餐点，也喜欢尝试各种各样的烹饪风格。\n音乐才能： 张三会弹奏吉他，享受音乐的世界，也喜欢在朋友和家人聚会时演奏音乐。\n编程技能： 他拥有出色的计算机编程技能，是一位优秀的软件工程师。",
                'interests': "徒步旅行： 张三热爱大自然，经常进行徒步旅行，探索山川河流，欣赏自然之美。\n阅读： 他是一位书虫，热衷于各种类型的书籍，特别是科幻小说和历史书籍。\n篮球： 张三喜欢打篮球，经常和朋友一起在周末参加篮球比赛。"}
zhangsan = Persona(name=zhangsan_dic['name'], persona=zhangsan_dic['persona'], skills=zhangsan_dic['skills'] ,interests=zhangsan_dic['interests'])


lisi_dic= {'name': "李四",
                'persona': "自信和坚定： 李四有坚定的信念，自信而果断，能够在压力下保持冷静。\n创造力： 他是一位富有创造力的思考者，常常能够提出新颖的想法和解决问题的方法。\n友善和关心他人： 李四非常关心他人，善待朋友和家人，经常主动帮助别人，建立了强大的社交网络。\n冒险精神： 他喜欢尝试新事物，勇于冒险，经常参加各种刺激的户外活动。",
                'skills': "绘画和艺术： 李四是一位出色的艺术家，擅长绘画和创作艺术作品，他的作品经常展示在当地画廊。\n瑜伽： 他是一名有经验的瑜伽修行者，经常进行冥想和身体训练，保持身心健康。\n写作： 李四热爱写作，擅长创作诗歌和散文，也有一些出版的短篇小说。\n户外冒险： 他喜欢登山、潜水和徒步旅行，经常在自然中寻找刺激和冒险。",
                'interests': "探索古迹： 李四热爱历史和文化，经常前往古迹和博物馆，探索人类历史的奥秘。\n音乐： 他是一位音乐爱好者，喜欢演奏吉他，也热衷于参加音乐节和音乐会。\n烹饪： 李四对烹饪有浓厚兴趣，经常烹制各种美食，特别喜欢尝试国际美食。"}
lisi = Persona(name=lisi_dic['name'], persona=lisi_dic['persona'], skills=lisi_dic['skills'] ,interests=lisi_dic['interests'])
associative_memory = ["在一个繁忙的都市街头，一名街头艺术家正在创作一幅令人惊叹的涂鸦艺术作品，吸引了路人的目光。突然，一位年轻的音乐家拿起他的吉他，开始演奏一首感人的原创歌曲，创造出一场意外的街头表演，将人们聚集在一起，创造了一刻充满艺术和音乐的魔幻时刻。",
                      "在一个宁静的湖畔小屋，一位退休的科学家突然收到一封神秘的信件，信中提到了一个失踪多年的研究合作伙伴。信件中附带了一张旧照片，一个未解之谜的地点，以及一条神秘的信息。这突如其来的消息将引发一段旅程，揭示了多年来的谜团和未解之谜。",
                      "在一个古老的城市，一座已废弃多年的废墟突然传来神秘的音乐。当当地居民前往调查时，他们发现一个不可思议的场景：一支看似来自古老时代的音乐团队在废墟中演奏，音乐回荡在城市中，令人陶醉。这不寻常的音乐会引发了关于时间旅行和神秘力量的疑问，城市陷入了充满奇迹的事件之中。",
                      "在一个宁静的农村小镇，一颗明亮的陨石坠落在一片麦田中，引起了当地居民的轰动。当科学家和农夫们赶到现场进行研究时，他们惊讶地发现，这颗陨石散发出神秘的光芒，对植物生长和周围环境产生了神奇的影响。小镇陷入了一场关于科学、奇迹和自然力量的调查之中，引发了深刻的思考和探索。",
]
zhangsan.asso_mem = associative_memory
lisi.asso_mem = associative_memory
lisi.get_summarized_asso_mem()

['事件一：街头艺术家和音乐家共同创造的魔幻时刻。',
 '事件二：神秘信件引发的旅程揭示未解之谜。',
 '事件三：废墟中的古老音乐团队引发的时间旅行疑问。',
 '事件四：陨石坠落引发的科学、奇迹和自然力量探索。']

## Perceive

In [5]:
### 在perceive过程中，agents先获取附近的场景信息
### loc是agents当前所处位置，attn_width是能见度
def get_nearby_events(maze, loc, attn_width):
    row, column = loc[0], loc[1]
    perceive = []
    for i in range(max(row-attn_width,0), min(9,row+attn_width)+1):
        for j in range(max(column-attn_width,0), min(9,column+attn_width)+1):
            if 'events' in maze[i][j]:
                for event in maze[i][j]['events']:
                    eve = f"场景：{world_dic[maze[i][j]['world']]}， 事件: " + event
                    perceive.append(eve)
    return perceive

## Retrieve

In [6]:
previous_evns = ["在一个古老的图书馆，一位年轻的研究员翻阅一本陈旧的书时，意外发现了一张被藏在书页间的古老地图。这张地图指示着一处未知的迷失之地，充满传说和谜题。激发了他的好奇心，研究员决定展开一场探险，与一队多样化的探险家一同前往，寻找这个神秘之地，解开隐藏其中的古代宝藏和谜题。",
                    "在一个宁静的山区村庄，突然爆发了一场罕见的气象现象，天空中出现了奇怪的光球，伴随着雷声和强烈的电磁波。居民们困惑不解，然后这些光球突然合成一个巨大的光柱，将村庄中心的一棵古老树包围，树上开始绽放出华丽的花朵。这一事件引发了全村的惊讶和研究，居民们开始寻求解释这一不可思议的奇迹。",
                    "在一个繁忙的都市中，一座老旧的建筑被拆除，工人们在废墟下发现了一座长时间失落的地下城市。这个地下城市完全保存了古老的建筑、壁画和文物，引发了考古学家和历史学家的广泛关注。随着调查的深入，他们逐渐揭示了这座城市的历史和目的，以及它如何与现代世界相互联系，成为一个引人入胜的谜题。",
                    "在一个宁静的海边小镇，一群年轻人在一次夜间沙滩露营活动中，意外发现了一个闪烁的海洋生物，它们发光的蓝色色彩在夜晚的海浪中熠熠生辉。这些生物被发现时引发了一场科学界的热烈兴趣，他们开始研究这些闪烁生物的生态和行为习性，而小镇居民也开始举办夜间海滩观赏活动，将这一自然奇观分享给更多人。这一不寻常的发现让小镇的夏季活动变得更加独特和引人入胜。",
                    "在一个宁静的乡村，当地居民突然发现一群奇怪的植物在一夜之间迅速生长，这些植物散发出迷人的香气。随着时间的推移，居民们开始注意到这些植物有治愈属性，可以用于传统医学。这一不寻常的自然发现引发了一场全民研究和探索的热潮，以及有关草药学和自然疗法的新兴兴趣。当地居民和科学家开始合作，以挖掘这些植物的潜力，并寻求将其用于医学和保健领域。",
                    "在一个遥远的沙漠地区，一支考古队伍在沙丘下发现了一座古老的城市废墟，这座城市被认为已经失落了数千年。当考古学家们进一步研究时，他们在废墟中发现了一座神秘的地下迷宫，里面藏有未解之谜、宝藏和精密的文物。这一发现引发了国际科研团队的兴趣，他们将共同努力解开这座城市和地下迷宫的谜团，揭示其丰富的历史和文化秘密。这一事件也将引发考古界和历史研究领域的突破性发展。",
                    "在一个古老的寺庙，一名修道士突然宣布，他已悟道，能够实现心灵投影。他邀请附近的居民前来经验这一奇迹，许多人前来寻求智慧和启示。在修道士的引导下，人们开始经历心灵旅程，探索内心的迷宫，这一事件将深刻地影响寺庙和附近社区的生活，带来精神和哲学的变革。同时，学者和科学家也前来研究这一现象，试图理解其背后的科学和神秘力量。",
                    "在一个偏远的山区小镇，当地居民突然开始报告关于神秘灯光和不明飞行物的目击事件。这些光点在夜空中移动，产生令人难以置信的色彩和图案。随着时间的推移，这些事件变得越来越频繁，吸引了全球的媒体和研究人员的关注。这一现象引发了广泛的科学研究和讨论，人们纷纷提出各种解释和理论，而小镇的居民也开始团结起来，创建了一个神秘灯光观测社区，以记录和研究这些令人惊奇的现象。这一事件将不仅改变小镇的命运，也将改变我们对宇宙的理解。",
                    "在一个宁静的湖泊边，一群垂钓者和游泳者突然目击到湖中出现了一只巨大的水生生物，被形容为类似传说中的“水怪”。这一惊人的发现迅速引发了媒体的广泛报道和探险者的前来，试图捕捉这一生物的证据。科学家也前来研究，以探索这一未解之谜。湖泊周边的小镇开始迎接了游客的涌入，这一事件让小镇焕发了生气，并引发了对自然界奥秘的浓厚兴趣。",
                    "在一个繁忙的都市公园，一名街头魔术师正在表演惊人的魔术表演，吸引了围观的人们。然而，当他试图让一只鸽子从他的鎏金帽子中飞出时，却突然消失了，让人目瞪口呆。随后，一位路过的老奶奶走近，轻轻敲了敲魔术师的帽子，鸽子突然重新出现，而老奶奶也开始展现出令人惊叹的魔法技能，点燃了一场不可思议的魔法表演，改变了众人的一天。",
                    "在一个宁静的村庄，居民们突然发现天空中出现了一架不明飞行物体，它发出闪烁的彩光并发出奇怪的声音。当村民们围拢过去观察时，他们发现这架飞行物体着陆后，一群友好的外星生物从中走出来，他们似乎是来地球进行文化交流的。这一不寻常的相遇引发了一场全球性的关注和探讨，人们开始探讨地球与外星生命的互动，以及未来的合作与发展。",
                    "在一个宁静的海滨小镇，一场未曾有过的天文现象出现了，一颗流星雨异常明亮，照亮了整个夜空。人们走出户外，一同观赏这壮观景象，但随着流星雨消逝，他们发现陨石残骸开始落下，引发了短暂的恐慌。然而，当科学家前来研究时，他们惊讶地发现这些陨石碎片中蕴含着神秘的信息和技术，这一事件引发了全球性的科学研究和国际合作，以解开这些陨石的秘密。"
                    ]

### 定义retrieve，返回搜索到的相关联的事件
def retrieve(previous_evns, perceived, top_k=2):
    events_prompt = ''
    for i in range(len(previous_evns)):
        event = previous_evns[i]
        events_prompt = events_prompt + f'事件: {event}'
    summarize_prompt = '情概括下列的事件:' + events_prompt + '每个事件用一句话概括。'
    summarized_prev_mem = get_response(summarize_prompt)
    summarized_prev_mem = [eve for eve in summarized_prev_mem.split('\n')]
    reterived_mem_dic, _, _ = get_revelant_past_mem(perceived=perceived, prev_mem=summarized_prev_mem, top_k=top_k)
    reterived_mem_idx = []
    for k in reterived_mem_dic.keys():
        for i in range(len(reterived_mem_dic[k])):
            if reterived_mem_dic[k][i] not in reterived_mem_idx:
                reterived_mem_idx.append(reterived_mem_dic[k][i])

    focused_event = [summarized_prev_mem[reterived_mem_idx[i]] for i in range(len(reterived_mem_idx))]
    return focused_event

def get_revelant_past_mem(perceived, prev_mem, top_k=2):
    sim_score = [[0 for _ in range(len(prev_mem))] for _ in range(len(perceived))]
    relevent_idx = {}
    perceived_emb = [get_embedding(perceived[i]) for i in range(len(perceived))]
    prev_mem_emb = [get_embedding(prev_mem[i]) for i in range(len(prev_mem))]
    for i in range(len(perceived)):
        for j in range(len(prev_mem)):
            sim_score[i][j] = cos_sim(perceived_emb[i], prev_mem_emb[j])
        relevent_idx[i], _ = get_top_k_idx(sim_score[i])
    return relevent_idx, perceived_emb, prev_mem_emb

def cos_sim(a, b):
## 返回一个值 0-1 
    return dot(a, b)/(norm(a)*norm(b))


def get_top_k_idx(scores, top_k=2):
    t = copy.deepcopy(scores)
    # 求m个最大的数值及其索引
    max_number = []
    max_index = []
    for _ in range(top_k):
        number = max(t)
        index = t.index(number)
        t[index] = 0
        max_number.append(number)
        max_index.append(index)
    return max_index, max_number

## Plan

In [7]:
def make_plan(person, perceived, focused_env):
    plan_prompt = (
                "你是沙盒世界的一员，你的信息是{person_info}。\n"
                "你现在正在面对一些场景\n"
                "{scenarios}\n"
                "你遇到的近期经历是\n"
                "{asso_mem}\n"
                "你的类似过往经历是\n"
                "{focused_env}\n"
                "现在请根据这些经历，制定你的下一步计划\n"
            )
    person_info = person.get_basic_info_prompt()
    #print(person_info)

    scenarios = ''
    for i in range(len(perceived)):
        event = perceived[i]
        scenarios = scenarios + f'{event}\n'
    #print(scenarios)

    asso_mem = ''
    summarized_asso_mem = person.get_summarized_asso_mem()
    for i in range(len(summarized_asso_mem)):
        event = summarized_asso_mem[i]
        asso_mem = asso_mem + f'{event}\n'
    #print(asso_mem)

    focused_env = ''
    for i in range(len(focused_env)):
        event = focused_env[i]
        asso_mem = asso_mem + f'{event}\n'
    #rint(focused_env)

    plan_pref = plan_prompt.format_map({'person_info':person_info, 'scenarios':scenarios, 'asso_mem':asso_mem, 'focused_env':focused_env})

    plan = get_response(plan_pref)
    return plan, plan_pref

## Reflect

In [9]:
### 根据做出的计划，做出一定的总结和思考
def reflect(person, plan_pref, plan):
    reflect_prompt = ("根据以下信息：\n\n\n"
                    "{plan_pref}"
                    "你制定了计划：\n\n\n"
                    "{plan}"
                    "请根据你获得的信息和你的计划做出总结和思考，以第一人称的口吻，只返回反思内容：\n\n")

    reflect_pref = reflect_prompt.format_map({'plan_pref':plan_pref, 'plan':plan})
    reflection = get_response(reflect_pref)
    person.asso_mem.append(reflection)
    return reflection

## 完整流程

In [10]:
maze = make_maze()
maze[1][2]['world'], maze[1][2]['events']

In [19]:
step_size = 5
current_loc = (5, 5)

for i in range(step_size):
    perceived = get_nearby_events(maze=maze, loc=current_loc, attn_width=1)
    print("+++++++++++ agents见到的场景 +++++++++++++++++++")
    print(perceived)
    focused_eve = retrieve(previous_evns=previous_evns, perceived=perceived)
    print("+++++++++++ agents过往记忆相关的场景 +++++++++++++++++++")
    print(focused_eve)
    plan, plan_pref = make_plan(person=zhangsan, perceived=perceived, focused_env=focused_eve)
    reflection = reflect(person=zhangsan, plan_pref=plan_pref, plan=plan)
    print("+++++++++++ agents对近期发生的事情的总结 +++++++++++++++++++")
    print(reflection)
    current_loc[0] = current_loc[0] + random.randrange(-1, 2)
    current_loc[1] = current_loc[1] + random.randrange(-1, 2)
    print("+++++++++++ agents现在的位置是 +++++++++++++++++++")
    print(current_loc)

+++++++++++ agents见到的场景 +++++++++++++++++++
['场景：乡村， 事件: 当地的农民在田间劳作时，发现一只受伤的小狐狸。他们决定将小狐狸带回家，并开始照顾它。小狐狸在农民的照顾下恢复了健康，并开始在农民家中生活。最终，小狐狸成为了农民家庭的一员，并与他们共享了许多美好的时光。', '场景：森林， 事件: 突然间，森林中响起了一声尖叫声。一只小鹿被一只饿狼紧追不舍，它拼尽全力奔跑，但狼越来越近了。就在这时，一只老鹰飞过来，将小鹿抓在了它的爪子里，飞走了。饿狼嚎叫着停在原地，失去了它的猎物。小鹿虽然受了惊吓，但幸免于难。', '场景：都市， 事件: 一位名叫李丽的办公室白领在赶地铁时，突然发现自己的手机不见了。她慌乱地四处寻找，但没有发现任何踪迹。最后，她只能无奈地放弃寻找，只好回家重新办理手机挂失。', '场景：港口小镇， 事件: 在港口小镇，一艘货船因为恶劣天气无法停靠码头，只能停泊在海边。突然一声巨响，货船的船体撞上了礁石，船上货物散落一地。港口小镇的渔民们闻讯赶来救援，其中一位渔民在拯救货物时不慎滑倒受伤。经过紧急处理，伤势得到控制。最终，船只得到修复，货物也得以顺利运往目的地。', '场景：沙漠， 事件: 在沙漠中，突然出现了一场强烈的沙尘暴。暴风雪般的风沙使得能见度极低，人们无法辨认方向。一位旅行者不慎迷失方向，走到了沙漠的边缘，陷入了危险之中。他不断地寻找出路，但是被风沙掩盖的地形和景物让他无法判断方向。最终，他在沙漠中迷路了。', '场景：沙漠， 事件: 在沙漠中，突然出现了一场沙尘暴。暴风雪般的沙子使得能见度降至极低，人们不得不停下脚步，等待沙尘暴过去。在这场沙尘暴中，一个旅行者和他的骆驼不幸迷失了方向。他们在沙漠中漫无目的地走着，直到骆驼因为缺水而崩溃倒地。旅行者不得不步行前行，直到他终于看到了一个绿洲，发现了救援。尽管他的骆驼已经死亡，但是他仍然幸存下来。', '场景：乡村， 事件: 一位老农民在田间耕作时，发现了一只受伤的小兔子。他决定把小兔子带回家，并用自己的方法照顾它。经过几天的照顾，小兔子的伤势逐渐好转，变得非常活泼可爱。最终，老农民决定将小兔子留在自己家中，作为家庭的一员。', '场景：森林， 事件: 在森林中，突然出现了一只巨大的熊，它正在向一对年轻夫妇走来。夫妇俩惊慌失措，不知所措。熊越来越近，

TypeError: 'str' object is not callable